## Surrogate data to test euler angle to direction transformation 

In [ ]:
# surrogate data to test 
surrogate_df = pd.DataFrame(columns=['angle_x','angle_y','angle_z'], index=np.arange(360)) # "direction_x","direction_y","direction_z","sin(angle_x)"

surrogate_df["angle_x"] = surrogate_df["angle_y"] = surrogate_df["angle_z"] = 0


surrogate_df["angle_x"] = range(0,360,1)
surrogate_df["angle_x"] = surrogate_df["angle_x"] # / 360 * 2 * np.pi
#surrogate_df["angle_y"] = range(200,0,-2)
#surrogate_df["angle_y"] = surrogate_df["angle_y"] # / 360 * 2 * np.pi

pitch = surrogate_df['angle_x']
yaw = surrogate_df['angle_y']
roll = surrogate_df['angle_z']
   
    
'''
surrogate_df['direction_x'] = -np.cos(yaw)*np.sin(pitch)*np.sin(roll)-np.sin(yaw)*np.cos(roll)
surrogate_df['direction_y'] = -np.sin(yaw)*np.sin(pitch)*np.sin(roll)+np.cos(yaw)*np.cos(roll)
surrogate_df['direction_z'] =  np.cos(pitch)*np.sin(roll)
'''

surrogate_df

In [ ]:
result_vec = pd.DataFrame(columns=['angle_x','angle_y','angle_z',"direction_x","direction_y","direction_z"])

for idx in range(len(surrogate_df)):
    
    rot_x = surrogate_df['angle_x'][idx] 
    rot_y = surrogate_df['angle_y'][idx]
    rot_z = surrogate_df['angle_z'][idx] 

    r = R.from_euler(seq='XYZ', angles=[rot_x, rot_y, rot_z], degrees=True) 

    rotvec = r.as_rotvec()

    rotvec_norm = rotvec/np.linalg.norm(rotvec)
    
    '''
    temp = pd.DataFrame(columns=['angle_x','angle_y','angle_z',"direction_x","direction_y","direction_z"])
    temp.loc[0,'angle_x'] = rot_x
    temp.loc[0,'angle_y'] = rot_y
    temp.loc[0,'angle_z'] = rot_z
    temp.loc[0, 'direction_x'] = rotvec_norm[0]
    temp.loc[0,'direction_y'] = rotvec_norm[1]
    temp.loc[0,'direction_z'] = rotvec_norm[2]
    '''
    
    #print([rot_x,rot_y,rot_z,rotvec_norm[0],rotvec_norm[1],rotvec_norm[2]])
    
    result_vec.loc[idx] = [rot_x,rot_y,rot_z,rotvec_norm[0],rotvec_norm[1],rotvec_norm[2]]

pd.set_option('display.max_rows', 200)
result_vec
    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(1,2,1, projection='3d')

x = result_vec['angle_x']
y = result_vec['angle_y']
z = result_vec['angle_z']

ax1.set_xlim([0,360])
ax1.set_ylim([0,360])
ax1.set_zlim([0,360])
ax1.set_title('Rotation Angle')

ax1.scatter(x, y, z, c='r', marker='o')

ax2 = fig.add_subplot(1,2,2, projection='3d')

x = result_vec['direction_x']
y = result_vec['direction_y']
z = result_vec['direction_z']

ax2.set_xlim([-1,1])
ax2.set_ylim([-1,1])
ax2.set_zlim([-1,1])
ax2.set_title('Rotation Direction')

ax2.scatter(x, y, z, c='r', marker='o')

## Old Calculation of yaw, pitch roll, likely incorrect

In [ ]:
pitch = current_df['car_rotation_angles.x']
yaw = current_df['car_rotation_angles.y']
roll = current_df['car_rotation_angles.z']

# First option
current_df['car_rotation_direction.x'] = -np.cos(yaw)*np.sin(pitch)*np.sin(roll)-np.sin(yaw)*np.cos(roll)
current_df['car_rotation_direction.y'] = -np.sin(yaw)*np.sin(pitch)*np.sin(roll)+np.cos(yaw)*np.cos(roll)
current_df['car_rotation_direction.z'] =  np.cos(pitch)*np.sin(roll)

# Second option
current_df['car_rotation_direction.x'] = np.cos(yaw)*np.cos(pitch)
current_df['car_rotation_direction.y'] = np.sin(yaw)*np.cos(pitch)
current_df['car_rotation_direction.z'] = np.sin(pitch)

## Check different ways of calculating direction from angles 

In [ ]:
from scipy.spatial.transform import Rotation as R

test_vecs = pd.DataFrame(columns=['X','Y','Z'])
idx = 0
for idx in range(len(ref_data_one_participant_single_segment)):
    
    carRot_x = ref_data_one_participant_single_segment['car_rotation_angles.x'][idx] 
    carRot_y = ref_data_one_participant_single_segment['car_rotation_angles.y'][idx] 
    carRot_z = ref_data_one_participant_single_segment['car_rotation_angles.z'][idx] 

    r = R.from_euler(seq='zxy', angles=[carRot_x, carRot_y, carRot_z], degrees=True) 

    rotvec = r.as_rotvec()

    rotvec_norm = rotvec/np.linalg.norm(rotvec)
    
    temp = pd.DataFrame(columns=['X','Y','Z'])
    
    temp.loc[0, 'X'] = rotvec_norm[0]
    temp.loc[0,'Y'] = rotvec_norm[1]
    temp.loc[0,'Z'] = rotvec_norm[2]
    
    test_vecs = test_vecs.append(temp)
    




In [ ]:
plt.plot(range(len(ref_data_one_participant_single_segment['car_rotation_direction.x'])),ref_data_one_participant_single_segment['car_rotation_direction.x'])

In [ ]:
plt.plot(range(len(test_vecs)),test_vecs.X)

## Extracting participant data

In [ ]:
one_participant_single_segment = measurement1.get_data(use_vanilla=False, filter_data=True, \
                 get_first_segment=False, after_event_type_only=[True,False], \
                 exclude_areas=[], exclude_segments=[], \
                 exclude_participants=['0d0d7bd79b1a48d4ad5e757cdbfc15c9']) 